In [1]:
import pandas as pd
import numpy as np
from typing import Optional
from datetime import date
import math
import pickle
from dataclasses import asdict, dataclass

In [74]:
# Удалить Дата исполнения заказа/договора* Месяц Год Дата поставки SWAP Эрикссон
expected_april = pd.read_excel('4.Ожидаемые поставки_Апрель2021.xlsx', header=3, sheet_name=0) #.drop(['Дата исполнения заказа/договора*', 'Месяц', 'Год', 'Дата поставки SWAP Эрикссон'], axis=1)
expected_april_set = set(expected_april.columns)

In [75]:
# Удалить длина-высота-ширина
expected_december = pd.read_excel('4.Ожидаемые поставки_Декабрь2020.xlsx', header=0, sheet_name=1) #.drop(['Длина', 'Ширина', 'Высота'], axis=1)
expected_december_set = set(expected_december.columns)

In [76]:
expected_january = pd.read_excel('4.Ожидаемые поставки_Январь2021.xlsx', header=0, sheet_name=1, skipfooter=7) #.drop(['Дата исполнения заказа/договора', 'Месяц', 'Год', 'Длина', 'Ширина', 'Высота'], axis=1)
expected_january_set = set(expected_january.columns)

In [86]:
intersected = expected_april_set & expected_december_set & expected_january_set
expected_april = expected_april[intersected]
expected_december = expected_december[intersected]
expected_january = expected_january[intersected]

In [89]:
concated_df = pd.concat([expected_april, expected_december, expected_january], axis=0, ignore_index=True)
with open('concated_df.pkl', 'wb') as f:
    pickle.dump(concated_df, f)

In [62]:
with open('concated_df.pkl', 'rb') as f:
    concated_df=pickle.load(f)
concated_df.head()
# len(concated_df)
# MSK: 101, К04, 107, К05
# SZ: 304
# Ural: 702

,Организация-получатель,Дата исполнения заказа/договора,Номер строки в заказе/релизе,В пути,Общий объём,Код позиции ЕНС,Наименование позиции ЕНС,Единица измерения,Полученное количество,Объём,Заказанное количество
0,101,13.05.2021,1,34,1.7,160.346.000041,"Блок внутренний IDU Huawei RTN 910AF, 2хGE RJ4...",штука,0,"0,05",34
1,101,13.05.2021,2,136,0.060928,249.022.000011,"Приемопередатчик оптический (eSFP, 1310нм, 1.2...",штука,0,0.000448,136
2,101,13.05.2021,3,8,0.16,161.346.000136,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,8
3,101,13.05.2021,4,8,0.16,161.346.000137,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,8
4,101,13.05.2021,5,9,0.18,161.346.000132,"Радиоблок внешний ODU Huawei RTN XMC-5D 18ГГц,...",штука,0,0.02,9


In [58]:
concated_df=concated_df.dropna(axis=0)
for idx, row in concated_df.iterrows():
    if isinstance(row['Общий объём'], float):
        continue
    concated_df.at[idx,'Общий объём'] = float(row['Общий объём'].replace(',', '.'))
grouped = concated_df.groupby(['Организация-получатель', 'Дата исполнения заказа/договора'])['Общий объём'].sum().reset_index()
msk = grouped.loc[grouped['Организация-получатель'].isin(['101', 'К04', '107', 'К05'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()
northwest = grouped.loc[grouped['Организация-получатель'].isin(['304'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()
ural = grouped.loc[grouped['Организация-получатель'].isin(['702'])].groupby('Дата исполнения заказа/договора')['Общий объём'].sum().reset_index()

groupe_dfs = {'msk': msk, 'northwest': northwest, 'ural': ural}
groupe_dfs

{'msk':     Дата исполнения заказа/договора Общий объём
 0               2020-12-06 00:00:00    7.125169
 1                        01.02.2021   24.063135
 2                        01.02.2022  141.643654
 3                        01.03.2021      1.0229
 4                        01.03.2022   14.470096
 ..                              ...         ...
 279                      31.08.2021    0.763344
 280                      31.10.2020      0.4008
 281                      31.12.2020       2.679
 282                      31.12.2021   36.551667
 283                      31.12.2022        0.27
 
 [284 rows x 2 columns],
 'northwest':     Дата исполнения заказа/договора Общий объём
 0               2021-01-22 00:00:00    14.63312
 1                        01.02.2021       1.185
 2                        01.02.2022    6.507779
 3                        01.03.2021  310.498313
 4                        01.04.2021   58.352702
 ..                              ...         ...
 196                  

In [63]:
with open('groupe_dfs.pkl', 'wb') as f:
    pickle.dump(groupe_dfs, f)